# API-Tensor
Using Tensorflow2 Machine Learning to automate API execution. API Tensors allow you to fully automate the API execution process.

This project aims at allowing developers to automate the API Automation process. Instead of writing, wiring, and maintaining API inputs, use Tensorflow and OpenAI Gym to 'learn' which data correlates to an API endpoints' input(s) -- powered by Artificial Intelligence.

In short, using OpenAI Gym, API-Tensor will repeatedly execute an endpoint using random values from a supplied list of possible values.



# Scenario

In this scenario, we will attempt to authenticate with our API with only knowing which table(s) in the database that required inputs come from.

In this case, we know that data comes from the 'users' and 'user_types' table(s).

Our endpoint requires two parameters (which do not match our table column names):

1. Username
2. Password

---

Users

| FIRST_NAME  | LAST_NAME   | USER_NAME   | AUTH_PASSWORD |
| ----------- | ----------- | ----------- | -----------   |
| ...         | ...         | ...         | ...           |

User_Types
| USER_TYPE  | USER_TYPE_ID |
| ----------- | ----------- |
| ...         | ...         |

Let's use OpenAI Gym to train our model to efficiently determine the best combination of data to get a successful endpoint response.

# Import Dependencies

In [1]:
import pandas as pd
import flatdict

from endpoint import APIEndpoint
from envs.api_env import APIEnv

# Goal

Let's start by defining our end-goal. This will let the model know that a certain combination of data is 0 or 1 (negative, positive).

In [2]:
def _goal_callback(resp: flatdict):
    try:
        # Endpoint response
        # Web Server JSON Response
        return resp['Success'] == True
    except:
        return False

# Gathering Data

In most cases, you are likely going to be using some type of relational database (since inputs require some sort of relation in order to guess). Any database or dataset should work so long as it can be converted to CSV or other Pandas supported format.

In this example, we will be using in-memory Pandas DataFrame. 

> Converting your SQL tables to Pandas DataFrame is extremely simple.

Users

| FIRST_NAME  | LAST_NAME   | USER_NAME   | AUTH_PASSWORD |
| ----------- | ----------- | ----------- | -----------   |
| ...         | ...         | ...         | ...           |

User_Types

| USER_TYPE  | USER_TYPE_ID |
| ----------- | ----------- |
| ...         | ...         |

In [3]:
data_users_columns = ['FIRST_NAME', 'LAST_NAME',
                      'USER_NAME', 'AUTH_PASSWORD']

data_acctype_columns = ['ACC_TYPE_DESC', 'ACC_TYPE_ID']

data_users = [
    ["Montray", "Davis", "@MontrayDavis", 'abc123'],
    ["Montray", "Davis", "@DemoUser1", 'abc456'],
    ["Montray", "Davis", "@DemoUser2", 'abc789']
]

data_account_types = [
    ['Administrator', 0],
    ['Standard User', 1]
]

db = pd.DataFrame(data_users, columns=data_users_columns)
db_acctypes = pd.DataFrame(data_account_types, columns=data_acctype_columns)

db = pd.concat([db, db_acctypes])

# Define our endpoint

Let's define our API endpoint.

> For now, API-Tensor only supports simple HTTP POST verbs.

In this scenario, we are using a simple post which requires a payload of the following format:

```
{
    Username: string,
    Password: string
}
```

You can find the working node-js sample in the external/web-server folder.

In [4]:
endpoint: APIEndpoint = APIEndpoint(
    "http://localhost:8081/login", ["Username", "Password"], "POST", {})


# Create the environment

In [5]:
env = APIEnv(endpoint, db, goal_callback=_goal_callback)

# Learning

This is where our model learns which inputs correlate with the provided data.

In [6]:
env.Learn(n_steps=1000, force_end_success=150, delay=0)

Epoch 1/15
1000/1000 [==============================] - 4s 3ms/step - loss: 0.0377 - accuracy: 0.9630
Epoch 2/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0310 - accuracy: 0.9690
Epoch 3/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0310 - accuracy: 0.9690
Epoch 4/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0310 - accuracy: 0.9690
Epoch 5/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0307 - accuracy: 0.9690
Epoch 6/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0264 - accuracy: 0.9690
Epoch 7/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0222 - accuracy: 0.9690
Epoch 8/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0200 - accuracy: 0.9690
Epoch 9/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0157 - accuracy: 0.9690
Epoch 10/15
1000/1000 [==============================] - 3s 3ms/step - loss: 0.011

# Load model and test

In [7]:
data_inputs = env.Run()

print(data_inputs)

36/36 [==============================] - 0s 3ms/step
Best Sequence: [2, 3, 1]
['USER_NAME', 'AUTH_PASSWORD']


# Successful API Calls

In [8]:
len([i for i in env.action_history if i[-1] == 1])

36